In [1]:
import requests
import pandas as pd

In [2]:
def get_crypto_list():
    url = f'https://web-api.coinmarketcap.com/v1/cryptocurrency/map'
    return requests.get(url).json()

In [3]:
currency_list = get_crypto_list()
currency_list_flat = [currency for currency in currency_list['data']]
currency_list_df = pd.DataFrame(currency_list_flat)

currency_list_df.head()

,id,name,symbol,slug,rank,is_active,first_historical_data,last_historical_data,platform
0,1,Bitcoin,BTC,bitcoin,1,1,2013-04-28T18:47:21.000Z,2021-10-20T13:09:02.000Z,None
1,2,Litecoin,LTC,litecoin,14,1,2013-04-28T18:47:22.000Z,2021-10-20T13:09:02.000Z,None
2,3,Namecoin,NMC,namecoin,776,1,2013-04-28T18:47:22.000Z,2021-10-20T13:09:02.000Z,None
3,4,Terracoin,TRC,terracoin,2009,1,2013-04-28T18:47:22.000Z,2021-10-20T13:09:02.000Z,None
4,5,Peercoin,PPC,peercoin,760,1,2013-04-28T18:47:23.000Z,2021-10-20T13:09:02.000Z,None


In [4]:
currency_list_df.loc[:, 'first_historical_data'] = pd.to_datetime(currency_list_df.loc[:, 'first_historical_data'])
currency_list_df.loc[:, 'last_historical_data'] = pd.to_datetime(currency_list_df.loc[:, 'last_historical_data'])
currency_list_df.set_index('id', inplace=True)

currency_list_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6650 entries, 1 to 12925
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   name                   6650 non-null   object             
 1   symbol                 6650 non-null   object             
 2   slug                   6650 non-null   object             
 3   rank                   6650 non-null   int64              
 4   is_active              6650 non-null   int64              
 5   first_historical_data  6650 non-null   datetime64[ns, UTC]
 6   last_historical_data   6650 non-null   datetime64[ns, UTC]
 7   platform               5311 non-null   object             
dtypes: datetime64[ns, UTC](2), int64(2), object(4)
memory usage: 467.6+ KB


In [5]:
def get_currency_quotes(currency_id, convert, start_date, end_date):
    url = f'https://web-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical?id={currency_id}&convert={convert}&time_start={start_date}&time_end={end_date}'
    return requests.get(url).json()

In [6]:
final_df = pd.DataFrame()

# Для начала получим только данные для первых 10 валют
cropped_list = currency_list_df.head(10)

for index, row in cropped_list.iterrows():
    start_date = row['first_historical_data'].strftime('%Y-%m-%d')
    end_date = row['last_historical_data'].strftime('%Y-%m-%d')
    
    data = get_currency_quotes(index, 'RUB', start_date, end_date)
    data_flat = [quote['quote']['RUB'] for quote in data['data']['quotes']]
    
    df = pd.DataFrame(data_flat)
    df.loc[:, 'symbol'] = row['symbol']
    df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'timestamp'])

    final_df = pd.concat((df, final_df), ignore_index=True)

final_df.shape

(29875, 8)

In [12]:
final_df.head()

,open,high,low,close,volume,market_cap,timestamp,symbol
0,2.167072,2.234794,1.593911,1.593911,0.0,3.076915e+06,2013-05-23 23:59:00+00:00,WDC
1,1.588879,1.791531,0.615499,0.834511,0.0,1.736600e+06,2013-05-24 23:59:00+00:00,WDC
2,0.833885,1.251767,0.457205,0.826868,0.0,1.861169e+06,2013-05-25 23:59:00+00:00,WDC
3,0.826905,1.032629,0.751398,0.803499,0.0,1.922421e+06,2013-05-26 23:59:00+00:00,WDC
4,0.450771,0.465826,0.380137,0.414453,0.0,1.946290e+06,2013-06-09 23:59:00+00:00,WDC


In [13]:
final_df.tail()

,open,high,low,close,volume,market_cap,timestamp,symbol
29870,4.094641e+06,4.458743e+06,4.060795e+06,4.372290e+06,3.675645e+12,8.239498e+13,2021-10-15 23:59:06+00:00,BTC
29871,4.373395e+06,4.420597e+06,4.273774e+06,4.322474e+06,2.431329e+12,8.145973e+13,2021-10-16 23:59:06+00:00,BTC
29872,4.322153e+06,4.381382e+06,4.202068e+06,4.374400e+06,2.063229e+12,8.244265e+13,2021-10-17 23:59:06+00:00,BTC
29873,4.374058e+06,4.454532e+06,4.280680e+06,4.417745e+06,2.710469e+12,8.326365e+13,2021-10-18 23:59:06+00:00,BTC
29874,4.418962e+06,4.565941e+06,4.371785e+06,4.553836e+06,2.867935e+12,8.583301e+13,2021-10-19 23:59:06+00:00,BTC


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29875 entries, 0 to 29874
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   open        29875 non-null  float64            
 1   high        29875 non-null  float64            
 2   low         29875 non-null  float64            
 3   close       29875 non-null  float64            
 4   volume      29875 non-null  float64            
 5   market_cap  29875 non-null  float64            
 6   timestamp   29875 non-null  datetime64[ns, UTC]
 7   symbol      29875 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(6), object(1)
memory usage: 1.8+ MB


In [8]:
final_df.to_csv('historical_cryptocurrency_prices_RUB.csv')